In [ ]:
import pygame

# --- Config ---
SCREEN_W, SCREEN_H = 600, 600
WORLD_W, WORLD_H   = 1000, 1000      # bigger than the screen
ROOM_W, ROOM_H = 200, 200
WALL_THINK = 5
DOOR_SIZE = 60
PLAYER_RADIUS = 15
PLAYER_SPEED  = 220                  # px / second
BG_COLOR      = (20, 20, 20)
GRID_COLOR    = (40, 40, 40)
WALL_COLOR    = (200, 200, 200)
PLAYER_COLOR  = (100, 200, 255)
FPS = 60
GRID_SIZE = 100

#------- Functions
def clamp(v, lo, hi):
    ''' keep player in world'''
    return max(lo, min(v, hi))



#-------- Classes
class Room:
    """
    A square room with 4 walls and a door gap in the middle of each wall.
    We precompute *wall rectangles* (in world coordinates) with the door cut out.
    """
    def __init__(self, rx, ry):
        self.rx = rx
        self.ry = ry
        self.rect = chunk_rect(rx, ry)
        self.wall_rects = self._build_walls()

    def _build_walls(self):
        r = self.rect
        t = WALL_THICK
        door = DOOR_SIZE

        walls = []

        # Helpers to add wall segments
        def add_rect(x, y, w, h):
            walls.append(pygame.Rect(x, y, w, h))

        # TOP wall (split around a centered door)
        top_y = r.top
        door_left = r.centerx - door // 2
        add_rect(r.left, top_y, door_left - r.left, t)  # left segment
        add_rect(door_left + door, top_y, r.right - (door_left + door), t)  # right segment

        # BOTTOM wall
        bottom_y = r.bottom - t
        add_rect(r.left, bottom_y, door_left - r.left, t)
        add_rect(door_left + door, bottom_y, r.right - (door_left + door), t)

        # LEFT wall
        left_x = r.left
        door_top = r.centery - door // 2
        add_rect(left_x, r.top, t, door_top - r.top)
        add_rect(left_x, door_top + door, t, r.bottom - (door_top + door))

        # RIGHT wall
        right_x = r.right - t
        add_rect(right_x, r.top, t, door_top - r.top)
        add_rect(right_x, door_top + door, t, r.bottom - (door_top + door))

        return walls






def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
    pygame.display.set_caption("Scrolling Camera Dungeon")
    clock = pygame.time.Clock()

    # Player starts in the middle of the world
    x, y = WORLD_W / 2, WORLD_H / 2

    running = True
    while running:
        dt = clock.tick(FPS) / 1000.0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # --- Input ---
        keys = pygame.key.get_pressed()
        dx = dy = 0
        if keys[pygame.K_a] or keys[pygame.K_LEFT]:
            dx -= 1
        if keys[pygame.K_d] or keys[pygame.K_RIGHT]:
            dx += 1
        if keys[pygame.K_w] or keys[pygame.K_UP]:
            dy -= 1
        if keys[pygame.K_s] or keys[pygame.K_DOWN]:
            dy += 1

        if dx and dy:  # normalize diagonals
            inv = 0.70710678
            dx *= inv
            dy *= inv

        # --- Update player in **world** coordinates ---
        x += dx * PLAYER_SPEED * dt
        y += dy * PLAYER_SPEED * dt

        # Keep player inside world bounds
        x = clamp(x, PLAYER_RADIUS, WORLD_W - PLAYER_RADIUS)
        y = clamp(y, PLAYER_RADIUS, WORLD_H - PLAYER_RADIUS)

        # --- Camera follows player, clamped to world ---
        cam_x = int(x - SCREEN_W / 2)
        cam_y = int(y - SCREEN_H / 2)
        cam_x = clamp(cam_x, 0, WORLD_W - SCREEN_W)
        cam_y = clamp(cam_y, 0, WORLD_H - SCREEN_H)

        # --- Draw ---
        screen.fill(BG_COLOR)

        # Draw a simple grid so scrolling is obvious
        first_vertical = (cam_x // GRID_SIZE) * GRID_SIZE
        first_horizontal = (cam_y // GRID_SIZE) * GRID_SIZE

        for gx in range(first_vertical, cam_x + SCREEN_W + GRID_SIZE, GRID_SIZE):
            sx = gx - cam_x
            pygame.draw.line(screen, GRID_COLOR, (sx, 0), (sx, SCREEN_H))
        for gy in range(first_horizontal, cam_y + SCREEN_H + GRID_SIZE, GRID_SIZE):
            sy = gy - cam_y
            pygame.draw.line(screen, GRID_COLOR, (0, sy), (SCREEN_W, sy))

        # World border (the giant "room")
        pygame.draw.rect(
            screen,
            WALL_COLOR,
            pygame.Rect(-cam_x, -cam_y, WORLD_W, WORLD_H),
            width=4
        )

        # Player (convert world -> screen by subtracting camera)
        pygame.draw.circle(
            screen,
            PLAYER_COLOR,
            (int(x - cam_x), int(y - cam_y)),
            PLAYER_RADIUS
        )

        pygame.display.set_caption(f"Scrolling Camera Dungeon | FPS: {clock.get_fps():.0f}")
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

: 